In [1]:
import numpy as np
import pygad
from pyswarm import pso
from sklearn.metrics import accuracy_score
from dbn.tensorflow import SupervisedDBNClassification
from tensorflow.keras.datasets import mnist

# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

x_test

ModuleNotFoundError: No module named 'dbn'

In [ ]:


# -------- Step 1: GA for Finding the Best DBN Architecture -------- #
def ga_fitness(solution, solution_idx):
    num_layers = int(solution[0])
    neurons_per_layer = int(solution[1])

    num_layers = max(1, min(5, num_layers))
    neurons_per_layer = max(10, min(500, neurons_per_layer))

    return num_layers, neurons_per_layer  # Return best architecture

ga_instance = pygad.GA(
    num_generations=5, num_parents_mating=3,
    fitness_func=ga_fitness, sol_per_pop=6,
    num_genes=2, init_range_low=1, init_range_high=500, gene_type=int
)
ga_instance.run()
best_architecture, _, _ = ga_instance.best_solution()

best_layers = int(best_architecture[0])
best_neurons = int(best_architecture[1])
print(f"Best DBN Architecture: {best_layers} layers, {best_neurons} neurons per layer")

# -------- Step 2: PSO for Fine-Tuning Training Parameters -------- #
def pso_fitness(params):
    learning_rate, batch_size = params
    learning_rate = max(0.0001, min(0.1, learning_rate))  # Limit learning rate range
    batch_size = max(16, min(128, int(batch_size)))  # Limit batch size range

    dbn = SupervisedDBNClassification(
        hidden_layers_structure=[best_neurons] * best_layers,
        learning_rate=learning_rate, batch_size=batch_size,
        n_epochs_rbm=10, n_iter_backprop=50, verbose=False
    )

    dbn.fit(x_train, y_train)
    y_pred = dbn.predict(x_test)
    return -accuracy_score(y_test, y_pred)  # PSO minimizes, so return negative accuracy

best_params, _ = pso(pso_fitness, [0.0001, 16], [0.1, 128])
best_lr, best_batch = best_params
print(f"Best Training Parameters: Learning Rate={best_lr}, Batch Size={int(best_batch)}")

# -------- Step 3: Train Final DBN with Optimized Parameters -------- #
final_dbn = SupervisedDBNClassification(
    hidden_layers_structure=[best_neurons] * best_layers,
    learning_rate=best_lr, batch_size=int(best_batch),
    n_epochs_rbm=10, n_iter_backprop=50, verbose=True
)

final_dbn.fit(x_train, y_train)
y_final_pred = final_dbn.predict(x_test)
final_accuracy = accuracy_score(y_test, y_final_pred)

print(f"Final DBN Accuracy: {final_accuracy}")

In [1]:
# UPDATE CODE 

In [2]:
!pip install deep-belief-network

ERROR: Could not find a version that satisfies the requirement deep-belief-network (from versions: none)
ERROR: No matching distribution found for deep-belief-network


In [8]:
import numpy as np
import pygad
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from deepbeliefnetwork import SupervisedDBNClassification
from pyswarm import pso  # Install with `pip install pyswarm`

ModuleNotFoundError: No module named 'deepbeliefnetwork'

In [7]:
# !pip install git+git://github.com/albertbup/deep-belief-network.git

In [ ]:


# -------- Step 1: Load & Preprocess Data -------- #
from sklearn.datasets import load_digits

# Load dataset (modify with your dataset)
data = load_digits()
X, y = data.data, data.target

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------- Step 2: GA for Finding the Best DBN Architecture -------- #
def ga_fitness(solution, solution_idx):
    num_layers = int(solution[0])
    neurons_per_layer = int(solution[1])

    num_layers = max(1, min(5, num_layers))  # Limit layers (1-5)
    neurons_per_layer = max(10, min(500, neurons_per_layer))  # Limit neurons (10-500)

    return num_layers, neurons_per_layer  # Return best architecture

ga_instance = pygad.GA(
    num_generations=5, num_parents_mating=3,
    fitness_func=ga_fitness, sol_per_pop=6,
    num_genes=2, init_range_low=1, init_range_high=500, gene_type=int
)

ga_instance.run()
best_architecture, _, _ = ga_instance.best_solution()

best_layers = int(best_architecture[0])
best_neurons = int(best_architecture[1])
print(f"Best DBN Architecture: {best_layers} layers, {best_neurons} neurons per layer")

# -------- Step 3: PSO for Fine-Tuning Training Parameters -------- #
def pso_fitness(params):
    learning_rate, batch_size = params
    learning_rate = max(0.0001, min(0.1, learning_rate))  # Limit learning rate range
    batch_size = max(16, min(128, int(batch_size)))  # Limit batch size range

    dbn = SupervisedDBNClassification(
        hidden_layers_structure=[best_neurons] * best_layers,
        learning_rate=learning_rate, batch_size=batch_size,
        n_epochs_rbm=10, n_iter_backprop=50, verbose=False
    )

    dbn.fit(x_train, y_train)
    y_pred = dbn.predict(x_test)
    return -accuracy_score(y_test, y_pred)  # PSO minimizes, so return negative accuracy

best_params, _ = pso(pso_fitness, [0.0001, 16], [0.1, 128])
best_lr, best_batch = best_params
print(f"Best Training Parameters: Learning Rate={best_lr}, Batch Size={int(best_batch)}")

# -------- Step 4: Train Final DBN with Optimized Parameters -------- #
final_dbn = SupervisedDBNClassification(
    hidden_layers_structure=[best_neurons] * best_layers,
    learning_rate=best_lr, batch_size=int(best_batch),
    n_epochs_rbm=10, n_iter_backprop=50, verbose=True
)

final_dbn.fit(x_train, y_train)
y_final_pred = final_dbn.predict(x_test)
final_accuracy = accuracy_score(y_test, y_final_pred)

print(f"Final DBN Accuracy: {final_accuracy:.4f}")


In [4]:
# import numpy as np
# import pandas as pd
# from sklearn.datasets import fetch_openml
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.neural_network import BernoulliRBM
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression


# # Load the dataset
# mnist = fetch_openml('mnist_784', version=1)
# X, y = mnist['data'], mnist['target']

# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42)

# # Preprocess the data by scaling it
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Initialize the RBM model
# rbm = BernoulliRBM(n_components=256, learning_rate=0.01, n_iter=20, verbose=1)
# # Initialize the logistic regression model
# logistic = LogisticRegression(max_iter=1000)
# # Create a pipeline that first extracts features using the RBM and then classifies with logistic regression
# dbn_pipeline = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
# # Train the DBN
# dbn_pipeline.fit(X_train_scaled, y_train)

# # Evaluate the model on the test set
# dbn_score = dbn_pipeline.score(X_test_scaled, y_test)
# print(f'DBN Classification score: {dbn_score}')


In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [8]:
# Load the dataset


In [7]:
X

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
